# LSTM网络情感二分类

In [1]:
import torch
from torch import nn
import tools
import pandas
import data_process


## 一、定义网络

自己写的embedding+lstm的网络，效果较差

In [2]:
class lstm(nn.Module):
    def __init__(self,num_embeddings,embedding_dim,hidden_size,device,num_layers=1,*args, **kwargs):
        super().__init__(*args, **kwargs)
        self.embedding = nn.Embedding(num_embeddings,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim,hidden_size,num_layers)
        self.ff = nn.Linear(hidden_size,2)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device
    def forward(self,X):
        h0 = torch.randn(self.num_layers,X.shape[0],self.hidden_size).to(self.device)
        c0 = torch.randn(self.num_layers,X.shape[0],self.hidden_size).to(self.device)
        X = self.embedding(X)
        X = X.transpose(0,1)
        output,_ = self.lstm(X,(h0,c0))
        return self.ff(output[-1])
        
        

改进的lstm，加了四个地方:
- lstm改成双向
- lstm加了dropout层
- 隐藏状态初始化为0
- 增加最大池化

In [3]:
class LSTM(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, device, num_layers=1, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, dropout=0.5, batch_first=True)#增加了droput层和双向
        self.ff = nn.Linear(2 * hidden_size, 2)  # 双向LSTM的输出需要乘以2
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device

    def forward(self, X):
        h0 = torch.zeros(self.num_layers * 2, X.shape[0], self.hidden_size).to(self.device)  # 隐藏状态初始化为0
        c0 = torch.zeros(self.num_layers * 2, X.shape[0], self.hidden_size).to(self.device)
        
        X = self.embedding(X)
        output, (hn, cn) = self.lstm(X, (h0, c0))

        # 使用最后一个时间步的输出进行分类
        output_pooled = torch.max(output, dim=1)[0]  # 这里使用最大池化
        return self.ff(output_pooled)


## 二、包装数据

In [4]:
data  = pandas.read_csv('./motionClassify.csv')
vocab = data_process.gen_vocab(data)
data_train  =  data_process.gen_dataset(data[:40000],vocab)
data_test = data_process.gen_dataset(data[40000:],vocab)
Batch_size = 64
train_iter = torch.utils.data.DataLoader(data_train,Batch_size,shuffle=True)
test_iter = torch.utils.data.DataLoader(data_test,Batch_size,shuffle=True)

## 三、训练参数设置

In [5]:


lr = 0.1
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda:0')
net1 = lstm(num_embeddings=len(vocab),embedding_dim=256,hidden_size=256,device=device)
optimizer1 = torch.optim.SGD(net1.parameters(),lr)
net2 = LSTM(num_embeddings=len(vocab),embedding_dim=256,hidden_size=256,device=device)
optimizer2 = torch.optim.SGD(net2.parameters(),lr)


c:\Users\ASUS\.conda\envs\dl\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


## 四、训练和测试

第一个网络训练和测试的结果

In [6]:
tools.train(net1,train_iter,device,optimizer1,criterion)

 16%|█▋        | 103/625 [00:05<00:25, 20.76it/s]

batch100,loss = 0.6900219917297363


 32%|███▏      | 202/625 [00:10<00:20, 20.54it/s]

batch200,loss = 0.7173640727996826


 48%|████▊     | 303/625 [00:14<00:15, 20.58it/s]

...

 80%|████████  | 501/625 [00:24<00:06, 20.17it/s]

batch500,loss = 0.6931577920913696


 96%|█████████▋| 603/625 [00:29<00:01, 20.85it/s]

batch600,loss = 0.6962697505950928


100%|██████████| 625/625 [00:30<00:00, 20.44it/s]


In [7]:
tools.test(net1,test_iter,device)

100%|██████████| 157/157 [00:02<00:00, 65.19it/s]

accuracy = 0.5005999803543091


准确率50%，接近自然概率，训练没有效果

第二个网络训练以及测试的结果

In [10]:
tools.train(net2,train_iter,device,optimizer2,criterion)

 16%|█▌        | 101/625 [00:08<00:41, 12.65it/s]

batch100,loss = 1.2120012044906616


 32%|███▏      | 201/625 [00:16<00:34, 12.39it/s]

batch200,loss = 0.844977080821991


 48%|████▊     | 301/625 [00:24<00:25, 12.48it/s]

...

 80%|████████  | 501/625 [00:40<00:10, 12.39it/s]

batch500,loss = 0.45665329694747925


 96%|█████████▌| 601/625 [00:48<00:01, 12.56it/s]

batch600,loss = 0.3577602207660675


100%|██████████| 625/625 [00:50<00:00, 12.40it/s]


In [11]:
tools.test(net2,test_iter,device)

100%|██████████| 157/157 [00:04<00:00, 32.38it/s]

accuracy = 0.7890999913215637


改进：可以增加训练的轮数(epoch)
可以做的任务：
- 可以把第一个网络一步步修改成第二个，看看到底是哪一个改进起了作用，实验发现仅仅改成双向网络效果还是差
- 可以在test函数里加入更多的benchmark，例如召回率（recall），F1-score，等等，见[春招算法题](./2024春招算法题.pdf)
- 对于第二个网络，可以通过增加训练轮数,改变学习率(lr)、词嵌入维度(embedding_size)、lstm的隐藏层神经元个数(hidden_size),优化器(optimizer)的种类等等参数